# Synthetic Data Generation Using RAGAS - RAG Evaluation with LangSmith

In the following notebook we'll explore a use-case for RAGAS' synthetic testset generation workflow!



- 🤝 BREAKOUT ROOM #1
  1. Use RAGAS to Generate Synthetic Data

- 🤝 BREAKOUT ROOM #2
  1. Load them into a LangSmith Dataset
  2. Evaluate our RAG chain against the synthetic test data
  3. Make changes to our pipeline
  4. Evaluate the modified pipeline

SDG is a critical piece of the puzzle, especially for early iteration! Without it, it would not be nearly as easy to get high quality early signal for our application's performance.

Let's dive in!

# 🤝 BREAKOUT ROOM #1

## Task 1: Dependencies and API Keys

We'll need to install a number of API keys and dependencies, since we'll be leveraging a number of great technologies for this pipeline!

1. OpenAI's endpoints to handle the Synthetic Data Generation
2. OpenAI's Endpoints for our RAG pipeline and LangSmith evaluation
3. QDrant as our vectorstore
4. LangSmith for our evaluation coordinator!

Let's install and provide all the required information below!

## Dependencies and API Keys:

> NOTE: DO NOT RUN THESE CELLS IF YOU ARE RUNNING THIS NOTEBOOK LOCALLY

In [1]:
#!pip install -qU ragas==0.2.10

In [2]:
#!pip install -qU langchain-community==0.3.14 langchain-openai==0.2.14 unstructured==0.16.12 langgraph==0.2.61 langchain-qdrant==0.2.0

### NLTK Import

To prevent errors that may occur based on OS - we'll import NLTK and download the needed packages to ensure correct handling of data.

In [3]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /home/olb/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/olb/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [4]:
import dotenv
dotenv.load_dotenv()

True

In [5]:
import os
import getpass
os.environ["LANGCHAIN_TRACING_V2"] = "true"
# print(os.getenv("OPENAI_API_KEY"))
# os.environ["LANGSMITH_API_KEY"] = getpass.getpass("LangSmith API Key:")

We'll also want to set a project name to make things easier for ourselves.

In [6]:
from uuid import uuid4

os.environ["LANGCHAIN_PROJECT"] = f"AIM - SDG - {uuid4().hex[0:8]}"

OpenAI's API Key!

In [7]:
#os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")

## Generating Synthetic Test Data

We wil be using Ragas to build out a set of synthetic test questions, references, and reference contexts. This is useful because it will allow us to find out how our system is performing.

> NOTE: Ragas is best suited for finding *directional* changes in your LLM-based systems. The absolute scores aren't comparable in a vacuum.

### Data Preparation

We'll prepare our data - which should hopefull be familiar at this point since it's our Loan Data use-case!

Next, let's load our data into a familiar LangChain format using the `DirectoryLoader`.

In [8]:
from langchain_community.document_loaders import DirectoryLoader
from langchain_community.document_loaders import PyMuPDFLoader


path = "data/"
loader = DirectoryLoader(path, glob="*.pdf", loader_cls=PyMuPDFLoader)
docs = loader.load()

### Knowledge Graph Based Synthetic Generation

Ragas uses a knowledge graph based approach to create data. This is extremely useful as it allows us to create complex queries rather simply. The additional testset complexity allows us to evaluate larger problems more effectively, as systems tend to be very strong on simple evaluation tasks.

Let's start by defining our `generator_llm` (which will generate our questions, summaries, and more), and our `generator_embeddings` which will be useful in building our graph.

### Unrolled SDG

In [9]:
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
generator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4.1-nano"))
generator_embeddings = LangchainEmbeddingsWrapper(OpenAIEmbeddings())

Next, we're going to instantiate our Knowledge Graph.

This graph will contain N number of nodes that have M number of relationships. These nodes and relationships (AKA "edges") will define our knowledge graph and be used later to construct relevant questions and responses.

In [10]:
from ragas.testset.graph import KnowledgeGraph

kg = KnowledgeGraph()
kg

KnowledgeGraph(nodes: 0, relationships: 0)

The first step we're going to take is to simply insert each of our full documents into the graph. This will provide a base that we can apply transformations to.

In [11]:
from ragas.testset.graph import Node, NodeType

### NOTICE: We're using a subset of the data for this example - this is to keep costs/time down.
for doc in docs[:20]:
    kg.nodes.append(
        Node(
            type=NodeType.DOCUMENT,
            properties={"page_content": doc.page_content, "document_metadata": doc.metadata}
        )
    )
kg

KnowledgeGraph(nodes: 20, relationships: 0)

Now, we'll apply the *default* transformations to our knowledge graph. This will take the nodes currently on the graph and transform them based on a set of [default transformations](https://docs.ragas.io/en/latest/references/transforms/#ragas.testset.transforms.default_transforms).

These default transformations are dependent on the corpus length, in our case:

- Producing Summaries -> produces summaries of the documents
- Extracting Headlines -> finding the overall headline for the document
- Theme Extractor -> extracts broad themes about the documents

It then uses cosine-similarity and heuristics between the embeddings of the above transformations to construct relationships between the nodes.

In [12]:
from ragas.testset.transforms import default_transforms, apply_transforms

transformer_llm = generator_llm
embedding_model = generator_embeddings

default_transforms = default_transforms(documents=docs, llm=transformer_llm, embedding_model=embedding_model)
apply_transforms(kg, default_transforms)
kg

Applying HeadlinesExtractor:   0%|          | 0/17 [00:00<?, ?it/s]

Applying HeadlineSplitter:   0%|          | 0/20 [00:00<?, ?it/s]

unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node


Applying SummaryExtractor:   0%|          | 0/31 [00:00<?, ?it/s]

Property 'summary' already exists in node '369fd5'. Skipping!
Property 'summary' already exists in node 'bf29e4'. Skipping!
Property 'summary' already exists in node '6c347c'. Skipping!
Property 'summary' already exists in node '373240'. Skipping!
Property 'summary' already exists in node 'e69df6'. Skipping!
Property 'summary' already exists in node 'd35deb'. Skipping!
Property 'summary' already exists in node '09e562'. Skipping!
Property 'summary' already exists in node 'a57b64'. Skipping!
Property 'summary' already exists in node 'c50946'. Skipping!
Property 'summary' already exists in node '01c4db'. Skipping!
Property 'summary' already exists in node '4f9863'. Skipping!
Property 'summary' already exists in node 'a2d62e'. Skipping!
Property 'summary' already exists in node 'ad8ff2'. Skipping!
Property 'summary' already exists in node '6d0450'. Skipping!


Applying CustomNodeFilter:   0%|          | 0/6 [00:00<?, ?it/s]

Applying [EmbeddingExtractor, ThemesExtractor, NERExtractor]:   0%|          | 0/43 [00:00<?, ?it/s]

Property 'summary_embedding' already exists in node '369fd5'. Skipping!
Property 'summary_embedding' already exists in node 'e69df6'. Skipping!
Property 'summary_embedding' already exists in node '373240'. Skipping!
Property 'summary_embedding' already exists in node 'd35deb'. Skipping!
Property 'summary_embedding' already exists in node 'ad8ff2'. Skipping!
Property 'summary_embedding' already exists in node 'bf29e4'. Skipping!
Property 'summary_embedding' already exists in node '6c347c'. Skipping!
Property 'summary_embedding' already exists in node 'a57b64'. Skipping!
Property 'summary_embedding' already exists in node 'a2d62e'. Skipping!
Property 'summary_embedding' already exists in node 'c50946'. Skipping!
Property 'summary_embedding' already exists in node '01c4db'. Skipping!
Property 'summary_embedding' already exists in node '09e562'. Skipping!
Property 'summary_embedding' already exists in node '4f9863'. Skipping!
Property 'summary_embedding' already exists in node '6d0450'. Sk

Applying [CosineSimilarityBuilder, OverlapScoreBuilder]:   0%|          | 0/2 [00:00<?, ?it/s]

KnowledgeGraph(nodes: 40, relationships: 480)

We can save and load our knowledge graphs as follows.

In [13]:
kg.save("loan_data_kg.json")
loan_data_kg = KnowledgeGraph.load("loan_data_kg.json")
loan_data_kg

KnowledgeGraph(nodes: 40, relationships: 480)

Using our knowledge graph, we can construct a "test set generator" - which will allow us to create queries.

In [14]:
from ragas.testset import TestsetGenerator

generator = TestsetGenerator(llm=generator_llm, embedding_model=embedding_model, knowledge_graph=loan_data_kg)

However, we'd like to be able to define the kinds of queries we're generating - which is made simple by Ragas having pre-created a number of different "QuerySynthesizer"s.

Each of these Synthetsizers is going to tackle a separate kind of query which will be generated from a scenario and a persona.

In essence, Ragas will use an LLM to generate a persona of someone who would interact with the data - and then use a scenario to construct a question from that data and persona.

In [15]:
from ragas.testset.synthesizers import default_query_distribution, SingleHopSpecificQuerySynthesizer, MultiHopAbstractQuerySynthesizer, MultiHopSpecificQuerySynthesizer

query_distribution = [
        (SingleHopSpecificQuerySynthesizer(llm=generator_llm), 0.5),
        (MultiHopAbstractQuerySynthesizer(llm=generator_llm), 0.25),
        (MultiHopSpecificQuerySynthesizer(llm=generator_llm), 0.25),
]

#### ❓ Question #1:

What are the three types of query synthesizers doing? Describe each one in simple terms.

#### ✅ Answer #1:
1. SingleHopSpecificQuerySynthesizer : Generates specific, fact-based queries that can be answered by retrieving information from a single document or source in one step.
2. MultiHopAbstractQuerySynthesizer: MultiHopAbstractQuerySynthesizer: Generates broad, interpretive queries that require retrieving information from multiple sources/documents and synthesizing them through multiple steps of higher-level reasoning to form comprehensive, analytical responses.
3. MultiHopSpecificQuerySynthesizer: Generates specific, fact-based queries that can be answered by retrieving information from multiple sources/documents and connecting them through multiple steps of reasoning.

Finally, we can use our `TestSetGenerator` to generate our testset!

In [16]:
testset = generator.generate(testset_size=10, query_distribution=query_distribution)
testset.to_pandas()

Generating personas:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Scenarios:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Samples:   0%|          | 0/11 [00:00<?, ?it/s]

,user_input,reference_contexts,reference,synthesizer_name
0,What are academic years in school?,"[Chapter 1 Academic Years, Academic Calendars,...",Academic years are defined as periods that may...,single_hop_specifc_query_synthesizer
1,Could you please explain the significance of t...,[Regulatory Citations Academic year minimums: ...,The regulation specified in 34 CFR 668.3(b) pe...,single_hop_specifc_query_synthesizer
2,What does Chapter 3 cover regarding clinical w...,[Inclusion of Clinical Work in a Standard Term...,Inclusion of clinical work in standard term pe...,single_hop_specifc_query_synthesizer
3,Is the Federal Work-Study program subject to p...,[Non-Term Characteristics A program that measu...,"No, the payment period is applicable to all Ti...",single_hop_specifc_query_synthesizer
4,Could you explain how Volume 7 relates to the ...,[both the credit or clock hours and the weeks ...,The provided context discusses the requirement...,single_hop_specifc_query_synthesizer
5,How do the requirements for instructional time...,[<1-hop>\n\nboth the credit or clock hours and...,The requirements for instructional time and pr...,multi_hop_abstract_query_synthesizer
6,How does the measurement of academic progress ...,[<1-hop>\n\nInclusion of Clinical Work in a St...,The inclusion of clinical work in a standard t...,multi_hop_abstract_query_synthesizer
7,How do overlapping courses across terms relate...,[<1-hop>\n\nInclusion of Clinical Work in a St...,Overlapping courses across terms are character...,multi_hop_abstract_query_synthesizer
8,Volume 2 and 7 info about credit hours and dis...,[<1-hop>\n\nboth the credit or clock hours and...,"The context explains that in Volume 2, it disc...",multi_hop_specific_query_synthesizer
9,Hwo do Volume 7 and Volume 8 help in understan...,[<1-hop>\n\nboth the credit or clock hours and...,Volume 7 provides guidance on calculating Pell...,multi_hop_specific_query_synthesizer


### Abstracted SDG

The above method is the full process - but we can shortcut that using the provided abstractions!

This will generate our knowledge graph under the hood, and will - from there - generate our personas and scenarios to construct our queries.



In [18]:
from ragas.testset import TestsetGenerator

generator = TestsetGenerator(llm=generator_llm, embedding_model=generator_embeddings)
dataset = generator.generate_with_langchain_docs(docs[:20], testset_size=10)

Applying HeadlinesExtractor:   0%|          | 0/17 [00:00<?, ?it/s]

Applying HeadlineSplitter:   0%|          | 0/20 [00:00<?, ?it/s]

unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node


Applying SummaryExtractor:   0%|          | 0/31 [00:00<?, ?it/s]

Property 'summary' already exists in node '0a1bae'. Skipping!
Property 'summary' already exists in node '3e8be9'. Skipping!
Property 'summary' already exists in node '0693c5'. Skipping!
Property 'summary' already exists in node 'c89d6d'. Skipping!
Property 'summary' already exists in node 'a07411'. Skipping!
Property 'summary' already exists in node 'f00fff'. Skipping!
Property 'summary' already exists in node '218aaa'. Skipping!
Property 'summary' already exists in node '1858ca'. Skipping!
Property 'summary' already exists in node '27fda3'. Skipping!
Property 'summary' already exists in node 'df917e'. Skipping!
Property 'summary' already exists in node 'f08a69'. Skipping!
Property 'summary' already exists in node '781935'. Skipping!
Property 'summary' already exists in node 'f25be9'. Skipping!
Property 'summary' already exists in node 'bec640'. Skipping!


...

Property 'summary_embedding' already exists in node '0693c5'. Skipping!
Property 'summary_embedding' already exists in node '3e8be9'. Skipping!
Property 'summary_embedding' already exists in node '1858ca'. Skipping!
Property 'summary_embedding' already exists in node 'f08a69'. Skipping!
Property 'summary_embedding' already exists in node '218aaa'. Skipping!
Property 'summary_embedding' already exists in node 'a07411'. Skipping!
Property 'summary_embedding' already exists in node '0a1bae'. Skipping!
Property 'summary_embedding' already exists in node 'bec640'. Skipping!
Property 'summary_embedding' already exists in node 'c89d6d'. Skipping!
Property 'summary_embedding' already exists in node 'f00fff'. Skipping!
Property 'summary_embedding' already exists in node 'f25be9'. Skipping!
Property 'summary_embedding' already exists in node 'df917e'. Skipping!
Property 'summary_embedding' already exists in node '27fda3'. Skipping!
Property 'summary_embedding' already exists in node '781935'. Sk

Applying [CosineSimilarityBuilder, OverlapScoreBuilder]:   0%|          | 0/2 [00:00<?, ?it/s]

Generating personas:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Scenarios:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Samples:   0%|          | 0/12 [00:00<?, ?it/s]

In [ ]:
dataset.to_pandas()

,user_input,reference_contexts,reference,synthesizer_name
0,What are Payment Periods in the context of aca...,"[Chapter 1 Academic Years, Academic Calendars,...",Payment Periods are part of the academic calen...,single_hop_specifc_query_synthesizer
1,What is 34 CFR 668.3(b) about?,[Regulatory Citations Academic year minimums: ...,34 CFR 668.3(b) pertains to weeks of instructi...,single_hop_specifc_query_synthesizer
2,What information does Volume 8 provide regardi...,[Inclusion of Clinical Work in a Standard Term...,Volume 8 discusses the inclusion of clinical w...,single_hop_specifc_query_synthesizer
3,What are Non-Term Characteristics in education...,[Non-Term Characteristics A program that measu...,Non-Term Characteristics refer to programs tha...,single_hop_specifc_query_synthesizer
4,H0w do disbursment timin in subscription-based...,[<1-hop>\n\nboth the credit or clock hours and...,The context explains that in clock-hour or non...,multi_hop_abstract_query_synthesizer
5,"So, like, if a program is credit-hour or clock...","[<1-hop>\n\nChapter 1 Academic Years, Academic...","According to the provided context, for credit-...",multi_hop_abstract_query_synthesizer
6,How does completing additional hours or weeks ...,[<1-hop>\n\nboth the credit or clock hours and...,Completing additional hours or weeks of instru...,multi_hop_abstract_query_synthesizer
7,disbursement timing in subscription programs a...,[<1-hop>\n\nboth the credit or clock hours and...,The context explains that in subscription-base...,multi_hop_abstract_query_synthesizer
8,Considering the detailed requirements outlined...,"[<1-hop>\n\nChapter 1 Academic Years, Academic...",The regulations in Volume 2 specify that each ...,multi_hop_specific_query_synthesizer
9,How do Appendix A and Appendix B guide the dis...,[<1-hop>\n\nDisbursement Timing in Subscriptio...,Appendix A provides guidance on disbursement t...,multi_hop_specific_query_synthesizer


We'll need to provide our LangSmith API key, and set tracing to "true".

# 🤝 BREAKOUT ROOM #2

## Task 4: LangSmith Dataset

Now we can move on to creating a dataset for LangSmith!

First, we'll need to create a dataset on LangSmith using the `Client`!

We'll name our Dataset to make it easy to work with later.

In [20]:
from langsmith import Client

client = Client()

dataset_name = "LoanSyntheticData"

langsmith_dataset = client.create_dataset(
    dataset_name=dataset_name,
    description="Loan Synthetic Data"
)

We'll iterate through the RAGAS created dataframe - and add each example to our created dataset!

> NOTE: We need to conform the outputs to the expected format - which in this case is: `question` and `answer`.

In [21]:
for data_row in dataset.to_pandas().iterrows():
  client.create_example(
      inputs={
          "question": data_row[1]["user_input"]
      },
      outputs={
          "answer": data_row[1]["reference"]
      },
      metadata={
          "context": data_row[1]["reference_contexts"]
      },
      dataset_id=langsmith_dataset.id
  )

## Basic RAG Chain

Time for some RAG!


In [22]:
rag_documents = docs

To keep things simple, we'll just use LangChain's recursive character text splitter!


In [23]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 500,
    chunk_overlap = 50
)

rag_documents = text_splitter.split_documents(rag_documents)

We'll create our vectorstore using OpenAI's [`text-embedding-3-small`](https://platform.openai.com/docs/guides/embeddings/embedding-models) embedding model.

In [24]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

As usual, we will power our RAG application with Qdrant!

In [26]:
from langchain_community.vectorstores import Qdrant

vectorstore = Qdrant.from_documents(
    documents=rag_documents,
    embedding=embeddings,
    location=":memory:",
    collection_name="Loan RAG"
)

In [27]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 10})

To get the "A" in RAG, we'll provide a prompt.

In [28]:
from langchain.prompts import ChatPromptTemplate

RAG_PROMPT = """\
Given a provided context and question, you must answer the question based only on context.

If you cannot answer the question based on the context - you must say "I don't know".

Context: {context}
Question: {question}
"""

rag_prompt = ChatPromptTemplate.from_template(RAG_PROMPT)

For our LLM, we will be using TogetherAI's endpoints as well!

We're going to be using Meta Llama 3.1 70B Instruct Turbo - a powerful model which should get us powerful results!

In [29]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4.1-mini")

Finally, we can set-up our RAG LCEL chain!

In [30]:
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain.schema import StrOutputParser

rag_chain = (
    {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
    | rag_prompt | llm | StrOutputParser()
)

In [31]:
rag_chain.invoke({"question" : "What kinds of loans are available?"})

'The kinds of loans available, based on the provided context, include:\n\n- Direct Subsidized Loans\n- Direct Unsubsidized Loans\n- Direct PLUS Loans (including student Federal PLUS Loans and parent Direct PLUS Loans)\n- Subsidized Federal Stafford Loans (under the now-ended FFEL Program)\n- Unsubsidized Federal Stafford Loans (under the now-ended FFEL Program)\n- Federal SLS Loans (under the now-ended FFEL Program)\n\nNote that new loans under the FFEL Program are no longer made since July 1, 2010. Direct Subsidized Loans are available only to undergraduate students, while graduate or professional students are eligible for Direct Unsubsidized Loans and Direct PLUS Loans.'

## LangSmith Evaluation Set-up

We'll use OpenAI's GPT-4.1 as our evaluation LLM for our base Evaluators.

In [32]:
eval_llm = ChatOpenAI(model="gpt-4.1")

We'll be using a number of evaluators - from LangSmith provided evaluators, to a few custom evaluators!

In [33]:
from langsmith.evaluation import LangChainStringEvaluator, evaluate

qa_evaluator = LangChainStringEvaluator("qa", config={"llm" : eval_llm})

labeled_helpfulness_evaluator = LangChainStringEvaluator(
    "labeled_criteria",
    config={
        "criteria": {
            "helpfulness": (
                "Is this submission helpful to the user,"
                " taking into account the correct reference answer?"
            )
        },
        "llm" : eval_llm
    },
    prepare_data=lambda run, example: {
        "prediction": run.outputs["output"],
        "reference": example.outputs["answer"],
        "input": example.inputs["question"],
    }
)

empathy_evaluator = LangChainStringEvaluator(
    "criteria",
    config={
        "criteria": {
            "empathy": "Is this response empathetic? Does it make the user feel like they are being heard?",
        },
        "llm" : eval_llm
    }
)

#### 🏗️ Activity #2:

Highlight what each evaluator is evaluating.

- `qa_evaluator`: It uses the "qa" LangChain evaluator to check the accuracy of the answer.
- `labeled_helpfulness_evaluator`: It uses our prompt. to instruct it to validate the generated answer using the "reference answe" and to check helpfulness to the user.
- `empathy_evaluator`: It uses our ptompt where we define "empathetic response", and validates the generated answer against this definition.

## LangSmith Evaluation

In [34]:
evaluate(
    rag_chain.invoke,
    data=dataset_name,
    evaluators=[
        qa_evaluator,
        labeled_helpfulness_evaluator,
        empathy_evaluator
    ],
    metadata={"revision_id": "default_chain_init"},
)

View the evaluation results for experiment: 'frosty-discussion-18' at:
https://smith.langchain.com/o/74b9e486-2e11-541b-b227-c9c177f98636/datasets/014fa91f-fac4-4c06-946c-0c4b896d3e30/compare?selectedSessions=9540551e-7a0f-4f06-b8aa-bb08487fcc23




0it [00:00, ?it/s]

,inputs.question,outputs.output,error,reference.answer,feedback.correctness,feedback.helpfulness,feedback.empathy,execution_time,example_id,id
0,"In the context of Appendx A and Appendx B, how...",Based on the provided context:\n\n- **Subscrip...,None,"According to the provided context, for clock-h...",0,0,0,5.107454,330c68fd-5abd-43b1-80bd-2cbbd13189b1,c38b9ff4-ad44-4df0-ac0c-305b79305fba
1,How do Volume 2 and Volume 8 relate to academi...,I don't know.,None,Volume 2 discusses the requirements for academ...,0,0,0,0.799370,5a62c16f-6a32-4df3-8efa-96728c722f5f,b67487a6-62e0-4bd1-9f73-7dd3d2009cf5
2,which chapters cover the 2 and 3 chapters in t...,I don't know,None,The context provided includes information from...,0,0,0,1.331820,c72c3c8d-627d-4540-84c8-5a2f36a0b656,96e01b18-1f71-4241-a1ac-80ea21ae9ff6
3,whats Volume 8,"Volume 8 is ""The Direct Loan Program."" It prov...",None,Volume 8 is referenced in the context as conta...,0,0,0,1.657558,4a31e316-5a79-4282-b116-c38792021a00,03ce9b0e-d5a6-434b-ab9f-ea5f12722334
4,How do the characteristics of nonstandard term...,"Based on the provided context, the inclusion o...",None,Nonstandard terms are defined as terms that do...,1,1,0,6.549854,a81b9e62-63ac-4038-801c-913d86d1487a,a71db2cc-3cbe-4d9b-ac96-617b887202e1
5,Considering the requirements for participation...,"Based on the provided context, clinical work c...",None,The inclusion of clinical work in standard ter...,0,0,0,3.575343,ea9ed0e6-4285-43ed-ae85-ccfe583c6c4b,761bd5a7-791f-49a8-a526-9e5a08002b08
6,How do academic calendars and weeks of instruc...,"Based on the context provided, academic calend...",None,Academic calendars specify the schedule of the...,1,1,0,8.233818,3748b611-9164-47c3-94f3-751153df05cb,bc692746-2b78-4ee9-92a3-d271481dfb9d
7,How do regulations and approval processes impa...,Regulations require that each academic year de...,None,Regulations and approval processes significant...,1,1,0,9.150868,d287fb12-7e36-4966-916a-5e64441b72cb,9d7777d4-ae1f-4e6e-abe5-a2a25d6b998e
8,How does the Title IV program define and handl...,"For Title IV purposes, payment periods are def...",None,The context states that payment periods are ap...,1,1,0,7.454585,f7005d0a-8597-4aad-a43f-5073515c4f2a,2799719b-e2cb-434e-aab7-0a47cff8ebc4
9,"In the context of Volume 8, how does the inclu...",Based on the provided context from Volume 8 an...,None,"Inclusion of clinical work in a standard term,...",1,0,0,4.189698,b8f7fbb3-4042-44e0-869c-f4c409ce67c3,8f55b383-f25e-46c4-8ab8-8715cefa3d3d


## Dope-ifying Our Application

We'll be making a few changes to our RAG chain to increase its performance on our SDG evaluation test dataset!

- Include a "dope" prompt augmentation
- Use larger chunks
- Improve the retriever model to: `text-embedding-3-large`

Let's see how this changes our evaluation!

In [35]:
EMPATHY_RAG_PROMPT = """\
Given a provided context and question, you must answer the question based only on context.

If you cannot answer the question based on the context - you must say "I don't know".

You must answer the question using empathy and kindness, and make sure the user feels heard.

Context: {context}
Question: {question}
"""

empathy_rag_prompt = ChatPromptTemplate.from_template(EMPATHY_RAG_PROMPT)

In [36]:
rag_documents = docs

In [37]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap = 50
)

rag_documents = text_splitter.split_documents(rag_documents)

#### ❓Question #2:

Why would modifying our chunk size modify the performance of our application?

#### ✅ Answer #2:
If the chunks are too small it might miss important context from neighboring chunks.
If the chunks are too large the LLM may get confused and miss the point.

In [44]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

#### ❓Question #3:

Why would modifying our embedding model modify the performance of our application?

#### ✅ Answer #3:

1. Accuracy: The number of dimensions can affect the content of the embeddings and that will affect the accuracy of the generated output.
1. Processing speed: The simpler the model the faster it will run but the outcome could be less accurate than a more powerful model.

In [39]:
vectorstore = Qdrant.from_documents(
    documents=rag_documents,
    embedding=embeddings,
    location=":memory:",
    collection_name="Loan Data for RAG"
)

In [40]:
retriever = vectorstore.as_retriever()

Setting up our new and improved DOPE RAG CHAIN.

In [41]:
empathy_rag_chain = (
    {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
    | empathy_rag_prompt | llm | StrOutputParser()
)

Let's test it on the same output that we saw before.

In [42]:
empathy_rag_chain.invoke({"question" : "What kinds of loans are available?"})

'Thank you for your question! Based on the context you’ve shared, there are several types of loans available to students and their parents to help cover educational costs:\n\n1. **Direct Subsidized Loans** – These are loans based on the student’s financial need and do not accumulate interest while the student is in school at least half-time.\n\n2. **Direct Unsubsidized Loans** – These loans are not based on financial need, and interest accrues during all periods.\n\n3. **Direct PLUS Loans** – These are loans that parents of dependent students can take out to cover the student’s cost of attendance. If the parent is ineligible, the student may qualify for additional unsubsidized loans.\n\nAlso, it’s important to know that students can receive a combination of Direct Subsidized and Direct Unsubsidized Loans, depending on their specific situation.\n\nI hope this helps clarify the loan options available to you! If you have any more questions or want to discuss your situation further, I’m he

Finally, we can evaluate the new chain on the same test set!

In [43]:
evaluate(
    empathy_rag_chain.invoke,
    data=dataset_name,
    evaluators=[
        qa_evaluator,
        labeled_helpfulness_evaluator,
        empathy_evaluator
    ],
    metadata={"revision_id": "empathy_rag_chain"},
)

View the evaluation results for experiment: 'best-machine-44' at:
https://smith.langchain.com/o/74b9e486-2e11-541b-b227-c9c177f98636/datasets/014fa91f-fac4-4c06-946c-0c4b896d3e30/compare?selectedSessions=b86c1317-162e-4e7e-8240-9c5dab32800e




0it [00:00, ?it/s]

,inputs.question,outputs.output,error,reference.answer,feedback.correctness,feedback.helpfulness,feedback.empathy,execution_time,example_id,id
0,"In the context of Appendx A and Appendx B, how...",Thank you for your thoughtful question. It’s c...,None,"According to the provided context, for clock-h...",1,0,1,5.706214,330c68fd-5abd-43b1-80bd-2cbbd13189b1,9891b21a-f22a-41fd-aca0-d56591f0f89c
1,How do Volume 2 and Volume 8 relate to academi...,Thank you for your thoughtful question. Based ...,None,Volume 2 discusses the requirements for academ...,1,1,1,4.551245,5a62c16f-6a32-4df3-8efa-96728c722f5f,69c160ec-9975-4715-ab8b-dee5450cbab9
2,which chapters cover the 2 and 3 chapters in t...,Thank you for your question. Based on the cont...,None,The context provided includes information from...,0,0,1,2.937452,c72c3c8d-627d-4540-84c8-5a2f36a0b656,8ffb440c-1eab-4eda-9986-a42cfad4b71b
3,whats Volume 8,Thank you for your question! Based on the cont...,None,Volume 8 is referenced in the context as conta...,1,0,1,3.464656,4a31e316-5a79-4282-b116-c38792021a00,9aaff3bc-2a0c-4f38-9efb-33df8c0d7d15
4,How do the characteristics of nonstandard term...,Thank you for your thoughtful question—it's cl...,None,Nonstandard terms are defined as terms that do...,1,1,1,4.948531,a81b9e62-63ac-4038-801c-913d86d1487a,bf656fd9-43e0-47fc-a2d7-0a310b6d62bb
5,Considering the requirements for participation...,Thank you for your thoughtful question—it’s cl...,None,The inclusion of clinical work in standard ter...,0,0,1,4.071962,ea9ed0e6-4285-43ed-ae85-ccfe583c6c4b,403844fb-21d8-4003-b98f-5229cf5a65bb
6,How do academic calendars and weeks of instruc...,Thank you for your thoughtful question. Based ...,None,Academic calendars specify the schedule of the...,1,1,1,3.810529,3748b611-9164-47c3-94f3-751153df05cb,19ce84ec-3fb2-40b8-8e50-7b0ebf1bec58
7,How do regulations and approval processes impa...,Thank you for your thoughtful question. It’s c...,None,Regulations and approval processes significant...,1,1,1,4.895873,d287fb12-7e36-4966-916a-5e64441b72cb,1560a531-cc09-404c-9bca-da7f7bf77857
8,How does the Title IV program define and handl...,Thank you for your thoughtful question. Based ...,None,The context states that payment periods are ap...,1,1,1,4.838575,f7005d0a-8597-4aad-a43f-5073515c4f2a,e22e0eda-9ca8-4650-af92-9a92a6bff4f9
9,"In the context of Volume 8, how does the inclu...",Thank you for your thoughtful question. It’s c...,None,"Inclusion of clinical work in a standard term,...",1,1,1,4.926909,b8f7fbb3-4042-44e0-869c-f4c409ce67c3,00b45519-8f55-4feb-9d70-ac7d7dc42d1e


#### 🏗️ Activity #3:

Provide a screenshot of the difference between the two chains, and explain why you believe certain metrics changed in certain ways.

<img src="assets/LangSmithEval_1.png" alt="LangSmith Evaluation" width="60%" /><img src="assets/LangSmithEval_1chunk.png" alt="LangSmith Evaluation" width=60%/>

#### ✅ Answer to 🏗️ Activity #3:

We are comparing the difference of the two chains before and after modifying the prompt by adding:  
 "You must answer the question using empathy and kindness, and make sure the user feels heard."  


 We observe that the "best_nachine-44" with the enpathy prompt, increased the "Correctness" a little,   
 gave points to the "Empathy" metric (all runs scored 100%), but made no difference in the "Helpfulness".